In [25]:
import configparser

# Read the configuration file
config = configparser.ConfigParser()
config.read('../config.ini')

# Access the API token from the configuration file
huggingfacehub_api_token = config.get('huggingface', 'api_token')
# Set your API Key from OpenAI
openai_api_key = config.get('openai', 'api_key')


In [ ]:
# Set your API Key from OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = PyPDFLoader('../database/MANIFESTO_OF_SURREALISMP1.pdf')
data = loader.load()
print(type(data))
print(data)

# Extract text from the PDF document
pdf_text = data.extract_text()

chunk_size = 200
chunk_overlap = 50

# Split the quote using RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap)

chunks = splitter.split_text(data_text) 

print(chunks[:5])

# Define an OpenAI embeddings model
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

persist_directory = 'persist_directory'

# Create the Chroma vector DB using the OpenAI embedding function; persist the database
vectordb = Chroma(    
    persist_directory=persist_directory,    
    embedding_function=embedding_model
    )

vectordb.add_texts(texts=chunks)


vectordb.persist()

#  retrieve embeddings for specific texts:

query_texts = chunks
embeddings = vectordb.retrieve_embeddings(query_texts)

num_embeddings = len(embeddings)
print("Number of embeddings:", num_embeddings)

In [28]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma


# Load the PDF document
loader = PyPDFLoader('../database/MANIFESTO_OF_SURREALISM.pdf')
data = loader.load()

# Define chunk size and overlap
chunk_size = 200
chunk_overlap = 50

# Initialize OpenAI embeddings model
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Initialize Chroma vector database
persist_directory = 'path/to/persist_directory'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

# Loop through each page of the PDF document
for document in data:
    # Extract text from the page
    page_text = document.page_content

    # Split the text into chunks
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_text(page_text)

    # Add texts to the vector database
    vectordb.add_texts(texts=chunks)

# Persist the vector database
vectordb.persist()




Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)


In [ ]:
query_texts = chunks
embeddings = vectordb.retrieve_embeddings(query_texts)

In [ ]:
query_texts = chunks
embeddings = embedding_model.embed(query_texts)
print(len(embeddings))